In [197]:
from requests import get
from bs4 import BeautifulSoup
import os

import pandas as pd
from datetime import date
import datetime

import preprocessing
import model
import wrangle

# Webscrapping

In [87]:
today = date.today()
print("Today's date:", today)

Today's date: 2020-07-22


In [93]:
month = today.month

In [94]:
day = today.day

In [95]:
year = today.year

In [132]:
airline = "aa"
flight_number = "1988"
flight_date = "08/02/2020"

In [135]:
datee = datetime.datetime.strptime(flight_date, "%m/%d/%Y")
datee

datetime.datetime(2020, 8, 2, 0, 0)

In [140]:
datee.day

2

In [96]:
url = f'https://www.flightstats.com/v2/flight-tracker/{airline}/{flight_number}?year={year}&month={month}&date={day}&flightId=1040676332'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
response = get(url, headers=headers)

In [97]:
soup = BeautifulSoup(response.content, 'html.parser')

In [98]:
airports = soup.find_all("a",class_="ticket__AirportLink-s1rrbl5o-10 cuIkvt")

In [99]:
dep_airport = airports[0].text
dep_airport

'CLT'

In [100]:
arr_airport = airports[1].text
arr_airport

'SAT'

In [111]:
crs_times = soup.find_all("div", class_="text-helper__TextHelper-s8bko4a-0 cCfBRT")

In [154]:
crs_dep_time = crs_times[0].text
crs_dep_time = int(crs_dep_time[0:5].replace(":",""))
crs_dep_time

1119

In [152]:
crs_arr_time = crs_times[2].text
crs_arr_time = int(crs_arr_time[0:5].replace(":", ""))
crs_arr_time

1301

# Preprocessing

In [123]:
features_for_modeling = ["op_carrier", "origin", "dest", "dest_airport_avg_delay", "carrier_avg_delay", "airport_avg_delay", "crs_dep_time", "crs_arr_time", "month", "day_of_week"]
target_variable = "is_delay"

In [34]:
# train_scaled, y_train, validate_scaled, y_validate, test_scaled, y_test = preprocessing.mvp_modeling_prep(modeling=True, features_for_modeling=features_for_modeling, target_variable=target_variable)

In [124]:
df = preprocessing.mvp_modeling_prep(features_for_modeling = features_for_modeling, target_variable = target_variable)

In [125]:
df.head()

,op_carrier,origin,dest,dest_airport_avg_delay,carrier_avg_delay,airport_avg_delay,crs_dep_time,crs_arr_time,month,day_of_week,is_delay
observation,,,,,,,,,,,
2018-01-01_UA_2429,UA,EWR,DEN,4.163647,6.260671,8.998358,1517,1745,1,0,False
2018-01-01_UA_2427,UA,LAS,SFO,9.162307,6.260671,4.242133,1115,1254,1,0,False
2018-01-01_UA_2424,UA,ORD,ALB,6.406991,6.260671,10.384382,630,922,1,0,True
2018-01-01_UA_2422,UA,ORD,OMA,3.662693,6.260671,10.384382,2241,14,1,0,False
2018-01-01_UA_2421,UA,IAH,LAS,3.966989,6.260671,3.795300,750,916,1,0,False


In [163]:
carrier_avg_delay = df[df.op_carrier == airline.upper()].carrier_avg_delay.mean()

In [175]:
dest_airport_avg_delay = df[df.dest == arr_airport].dest_airport_avg_delay.mean()

In [174]:
airport_avg_delay = df[df.origin == dep_airport].airport_avg_delay.mean()

In [177]:
pd.DataFrame({"carrier_avg_delay":carrier_avg_delay, "dest_airport_avg_delay": dest_airport_avg_delay, "airport_avg_delay":airport_avg_delay, "crs_dep_time": crs_dep_time, "crs_arr_time": crs_arr_time, "month": datee.month, "day_of_week":datee.day}, index=[f"{flight_date}_{airline.upper()}_{flight_number}"])

,carrier_avg_delay,dest_airport_avg_delay,airport_avg_delay,crs_dep_time,crs_arr_time,month,day_of_week
08/02/2020_AA_1988,6.861925,5.586935,7.573031,1119,1301,8,2


# Model

In [195]:
features_for_modeling = ["dest_airport_avg_delay", "carrier_avg_delay", "airport_avg_delay", "crs_dep_time", "crs_arr_time", "month", "day_of_week"]
target_variable = "is_delay"

In [ ]:
df = wrangle.prep_flight_data()
df = preprocessing.to_date_time(df)
df = preprocessing.create_new_features(df)
df = preprocessing.create_target_variable(df)

df = df[df.origin == dep_airport]

features_for_modeling += ["observation"]
features_for_modeling += [target_variable]

df_modeling = df[features_for_modeling]

df_modeling = df_modeling.set_index("observation")


train, validate, test = preprocessing.split_data(df_modeling)

X_train = train.drop(columns=target_variable)
y_train = train[target_variable]
X_validate = validate.drop(columns=target_variable)
y_validate = validate[target_variable]
X_test = test.drop(columns=target_variable)
y_test = test[target_variable]

scaler, train_scaled, validate_scaled, test_scaled = preprocessing.min_max_scaler(X_train, X_validate, X_test)

In [ ]:
knn, y_pred = model.run_knn(train_scaled, y_train)